In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals

In [ ]:
#pip install tensorflow
import tensorflow as tf

In [ ]:
import pandas as pd
import numpy as np
import nltk
from zipfile import ZipFile
df = pd.read_csv("https://raw.githubusercontent.com/huynhthanh98/AML-2022/main/Lab-04/spam_detection.csv")
df

,Text,Label,y
0,"Go until jurong point, crazy.. Available only ...",ham,0
1,Ok lar... Joking wif u oni...,ham,0
2,Free entry in 2 a wkly comp to win FA Cup fina...,spam,1
3,U dun say so early hor... U c already then say...,ham,0
4,"Nah I don't think he goes to usf, he lives aro...",ham,0
...,...,...,...
5569,This is the 2nd time we have tried 2 contact u...,spam,1
5570,Will Ã¼ b going to esplanade fr home?,ham,0
5571,"Pity, * was in mood for that. So...any other s...",ham,0
5572,The guy did some bitching but I acted like i'd...,ham,0


In [ ]:
nltk.download('punkt')
texts = df["Text"].to_list()
texts = [text.lower() for text in texts ]          
tokenized_texts = [nltk.tokenize.word_tokenize(text) for text in texts]    

print(tokenized_texts[0])

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\DangKhoa\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


['go', 'until', 'jurong', 'point', ',', 'crazy', '..', 'available', 'only', 'in', 'bugis', 'n', 'great', 'world', 'la', 'e', 'buffet', '...', 'cine', 'there', 'got', 'amore', 'wat', '...']


In [ ]:
import io
import numpy as np
def load_word_embeddings(fname):
    fin = io.open(fname, 'r', encoding='utf-8', newline='\n', errors='ignore')
    vocab, matrix = [], []
    i=0
    for line in fin:
        tokens = line.rstrip().split(' ')
        vocab.append(tokens[0])
        matrix.append(list(map(float, tokens[1:])))
    return vocab, np.asarray(matrix)

In [ ]:
vocab, matrix = load_word_embeddings("glove.6B.300d.txt")

In [ ]:
## Gán các mã
__PADDED_INDEX__ = 0   
__UNKNOWN_WORD__ = 1    

In [ ]:
# Tạo một dictionary
word_to_index = {word: index+2 for index, word in enumerate(vocab)}

In [ ]:
embedding_matrix = np.pad(matrix, [[2,0],[0,0]], mode='constant', constant_values =0.0)
print(embedding_matrix.shape)

(400002, 300)


In [ ]:
## Bây giờ ta sẽ chuyển data spam dection thành các mã số
import tensorflow as tf

X = []
for text in tokenized_texts:
    cur_text_indices = []
    for word in text:
        if word in word_to_index:
            cur_text_indices.append(word_to_index[word])    ## map từ word sang index
        else:
            cur_text_indices.append(__UNKNOWN_WORD__)       ## gán unknown cho từ không có trong bộ glove
    X.append(cur_text_indices)

## pad data cho có cùng chiều dài
X = tf.keras.preprocessing.sequence.pad_sequences(sequences=X,       # sequences: list các câu có độ dài không bằng nhau
                                                  padding='post')    # vị trí pad là 'pre' (trước) hoặc 'post' (sau)

y = df['y'].values   ## Label của bài toán, 0 là không phải spam, 1 là spam

In [ ]:
## Chia data
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size= 0.2, random_state =0)

# Mô Hình 

In [ ]:
from tensorflow.keras.layers import Input, Embedding, LSTM, Bidirectional, Dense
from tensorflow.keras.models import Model

inputs = Input(shape=(None,))                   

embed = Embedding(input_dim=embedding_matrix.shape[0],   
                 output_dim=embedding_matrix.shape[1],   
                  embeddings_initializer = tf.keras.initializers.Constant(value=embedding_matrix),  ## Khởi tạo cho embedding bằng ma trận có sẵn
                  trainable= True,                       
                 mask_zero=True)(inputs)                 

lstm = LSTM(units=100,                          ## units: kích thước của hidden_state trong LSTM
            return_sequences=False)(embed)      ## return_sequences: LSTM trả về toàn bộ  hay là trả về hidden_state cuối cùng

dense = Dense(units=2, activation='softmax')(lstm)
model = Model(inputs=inputs,
              outputs=dense)

model.compile(optimizer='adam',
             loss='sparse_categorical_crossentropy',
             metrics=['accuracy'])
model.summary()

Model: "model_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_3 (InputLayer)        [(None, None)]            0         
                                                                 
 embedding_2 (Embedding)     (None, None, 300)         120000600 
                                                                 
 lstm_2 (LSTM)               (None, 100)               160400    
                                                                 
 dense_2 (Dense)             (None, 2)                 202       
                                                                 
Total params: 120,161,202
Trainable params: 120,161,202
Non-trainable params: 0
_________________________________________________________________


In [ ]:
mc = tf.keras.callbacks.ModelCheckpoint(filepath="lstm_spam.h5", 
                                     monitor='val_loss',
                                     mode='min', 
                                     verbose=0, 
                                     save_best_only=True)
# Train
model.fit(X_train, y_train, 
          validation_data=(X_valid, y_valid),
         epochs=10,
          batch_size=80,
         callbacks=[mc])



Epoch 1/10
56/56 [==============================] - 203s 4s/step - loss: 0.1734 - accuracy: 0.9379 - val_loss: 0.0835 - val_accuracy: 0.9749
Epoch 2/10
56/56 [==============================] - 203s 4s/step - loss: 0.0492 - accuracy: 0.9856 - val_loss: 0.0560 - val_accuracy: 0.9839
Epoch 3/10
56/56 [==============================] - 192s 3s/step - loss: 0.0242 - accuracy: 0.9928 - val_loss: 0.0437 - val_accuracy: 0.9910
Epoch 4/10
56/56 [==============================] - 201s 4s/step - loss: 0.0137 - accuracy: 0.9960 - val_loss: 0.0431 - val_accuracy: 0.9883
Epoch 5/10
56/56 [==============================] - 114s 2s/step - loss: 0.0076 - accuracy: 0.9978 - val_loss: 0.0584 - val_accuracy: 0.9874
Epoch 6/10
56/56 [==============================] - 114s 2s/step - loss: 0.0043 - accuracy: 0.9987 - val_loss: 0.0533 - val_accuracy: 0.9901
Epoch 7/10
56/56 [==============================] - 116s 2s/step - loss: 0.0050 - accuracy: 0.9969 - val_loss: 0.0569 - val_accuracy: 0.9794
Epoch 8/10
56

In [ ]:
# Test
model.load_weights("lstm_spam.h5")
_, val_acc = model.evaluate(X_valid, y_valid)
print("Accuracy on valid: ", val_acc)

35/35 [==============================] - 2s 69ms/step - loss: 0.0431 - accuracy: 0.9883
Accuracy on valid:  0.9883407950401306


In [ ]:
from keras.preprocessing.text import Tokenizer

def Model_Predict(sentence):
    # Creating a tokenizer
    tokenizer = Tokenizer(lower=True)

    # Building word indices
    tokenizer.fit_on_texts(sentence)

    # Tokenizing sentences
    sentences_test = tokenizer.texts_to_sequences(sentence)
    padded = tf.keras.preprocessing.sequence.pad_sequences(sequences=sentences_test,  padding='post', truncating = "post")
    
    pred = model.predict(padded)
    for i in range(0,1):
        for j in range(0,2):
            if pred[i][j] >= 0.5:
                return True
            else:
                return False

In [ ]:
#Testing

In [ ]:
sentence_1 = ["wanna ask something? just send me a mess"]
sentence_2 = ["Urgent! You have won our competition's prize!! Please call us now."]
sentence_3 = ["Call me to get a free holiday now"]

In [ ]:
model_predict(sentence_1)

1/1 [==============================] - 2s 2s/step


True

In [ ]:
model_predict(sentence_2)

1/1 [==============================] - 1s 1s/step


False

In [ ]:
model_predict(sentence_3)

1/1 [==============================] - 0s 25ms/step


True